In [314]:
import altair as alt
import mysql.connector as mysql
from datetime import datetime
import time
import pandas as pd
import json
import numpy as np

In [315]:
fake_data = True  #set this to false to use data from the DB
config_file = './saraSqlConfig.json'
username = 'mash_aya'
num_days = 14

In [316]:
%%javascript
//toggle scrolling for jupyter notebooks.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [317]:
def getSqlConfigFromJSON(configFileName):
    """
    Loads full mysql connect object from database returns the value as a JSON object. Format of the JSON object 
    is the following:
    
        {
            "host": "hostname.com",
            "port": 99999,
            "user": "root",
            "passwd": "password",
            "database": "database_or_schema_name"
        }
        
    """
    
    with open(configFileName) as f:
        mysqlConnectObject = json.load(f)
        
    return mysqlConnectObject

mysqlConnectObject = getSqlConfigFromJSON(config_file)
db = mysql.connect(
    host = mysqlConnectObject["DB_HOST"],
    port = mysqlConnectObject["DB_PORT"],
    user = mysqlConnectObject["DB_USER"],
    passwd = mysqlConnectObject["DB_PASSWORD"],
    database = "HarvardDev",
    )

def get_SQL_data(command):
    cursor = db.cursor()
    cursor.execute(command)
    returnedData = cursor.fetchall()
    
    return returnedData

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'ec2-54-91-131-166.compute-1.amazonaws.com' (111)

In [318]:
#DB data
def get_mood(username = 'mash_aya', num_days = 7):
    
    sql_command = "SELECT when_inserted, json_answer FROM HarvardDev.harvardSurvey " 
    sql_command = sql_command + "where user_id=\""+username +"\" and when_inserted > NOW() - INTERVAL " + str(num_days) + " DAY "
    sql_command =  sql_command + " order by survey_completion_time desc;"
    
    returnedData = get_SQL_data(sql_command)
    
    #reformat data
    dates = []
    mood1=[]
    mood2=[]
    for row in returnedData:
        date = row[0]
        row_json=json.loads(row[1])
        try:
            dates.append(date)
            mood1.append(float(row_json['QMood'].split(":")[0]))
            mood2.append(float(row_json['QMood'].split(":")[0]))
        except:
            print(str(date) + " no mood")
    print(mood1)       
    return mood1,mood2,dates

def get_conc(username = 'mash_aya', num_days = 7):

    sql_command = "SELECT when_inserted, json_answer FROM HarvardDev.harvardSurvey " 
    sql_command = sql_command + "where user_id=\""+username +"\" and when_inserted > NOW() - INTERVAL " + str(num_days) + " DAY "
    sql_command =  sql_command + " order by survey_completion_time desc;"

    returnedData = get_SQL_data(sql_command)

    conc=[]
    xdates = []
    for row in returnedData:
        date = row[0]
        row_json=json.loads(row[1])
        try:
            conc.append(row_json['Q2'])
            xdates.append(row[0])
        except:
            print(str(date) + " no concentration level")
            
    return conc, xdates

def conc_to_int(arr): #from Philip; Convert labels to numerical
    new_list=[]
    for i in arr:
        if i == ' Rarely/Never':
            new_list.append(1)
        elif i == ' Occasionally':
            new_list.append(2)
        elif i == ' Often':
            new_list.append(3)
        elif i == ' Almost Always/Always':
            new_list.append(4)
    return new_list 


def get_good_day(username = 'mash_aya', num_days = 7):

    sql_command = "SELECT when_inserted, json_answer FROM HarvardDev.harvardSurvey " 
    sql_command = sql_command + "where user_id=\""+username +"\" and when_inserted > NOW() - INTERVAL " + str(num_days) + " DAY "
    sql_command =  sql_command + " order by survey_completion_time desc;"

    returnedData = get_SQL_data(sql_command)

    good_day=[]
    xdates = []
    for row in returnedData:
        date = row[0]
        row_json=json.loads(row[1])
        try:
            good_day.append(row_json['Q5'])
            xdates.append(row[0])
        except:
            print(str(date) + " no good day given")
            
    return good_day, xdates

get_good_day()

OperationalError: MySQL Connection not available.

In [319]:
if fake_data == True:
    
    #mood
    mood1 = np.random.uniform(-5,5,num_days)
    mood2 = np.random.uniform(-10,0,num_days)    
    mood_dates = pd.date_range(start='11/1/2020', periods=num_days)
    
    #concentration
    concs_options = [' Often',' Rarely/Never',' Occasionally',' Almost Always/Always']
    concs = np.random.choice(concs_options,num_days,replace=True)
    conc_dates = mood_dates
    
    #good_day
    gd_options = ['Yes', 'No']
    gd_data = np.random.choice(gd_options, num_days, replace = True)
    gd_dates = mood_dates
    
else:
    mood1,mood2,mood_dates = get_mood(username,num_days)
    concs,conc_dates = get_conc(username = username, num_days = num_days)
    gd_data, gd_dates = get_good_day(username = username, num_days = num_days)

#make lists in to pandas df    
mood_sum = [x+y for x,y in zip(mood2,mood2)]
mood_df = pd.DataFrame(list(zip(mood_dates,mood1,mood2,mood_sum)), \
        columns = ['date','mood1','mood2','mood_sum'])
xconc = conc_to_int(concs)
conc_df = pd.DataFrame({'conc':xconc,'conc_label':concs, 'date':conc_dates})

gd_df = pd.DataFrame({'good_day': gd_data, 'date': gd_dates})
    
#add column for reformatted date
if fake_data == True:
    mood_df['short_date'] = mood_df['date'].dt.strftime('%Y-%m-%d') #reformat dates for labels
    conc_df['short_date'] = conc_df['date'].dt.strftime('%Y-%m-%d') #reformat dates for labels
    gd_df['short_date'] = gd_df['date'].dt.strftime('%Y-%m-%d') #reformat dates for labels
else:
    mood_df['short_date'] = mood_df['date'].str.slice(0,10) 
    conc_df['short_date'] = mood_df['date'].str.slice(0,10) 
    gd_df['short_date'] = gd_df['date'].str.slice(0,10) 
    
print(gd_df) 

   good_day       date  short_date
0        No 2020-11-01  2020-11-01
1       Yes 2020-11-02  2020-11-02
2       Yes 2020-11-03  2020-11-03
3       Yes 2020-11-04  2020-11-04
4       Yes 2020-11-05  2020-11-05
5       Yes 2020-11-06  2020-11-06
6       Yes 2020-11-07  2020-11-07
7        No 2020-11-08  2020-11-08
8        No 2020-11-09  2020-11-09
9        No 2020-11-10  2020-11-10
10      Yes 2020-11-11  2020-11-11
11       No 2020-11-12  2020-11-12
12       No 2020-11-13  2020-11-13
13      Yes 2020-11-14  2020-11-14


In [320]:
#color scheme for all plots 
#https://htmlcolorcodes.com/color-picker/
""""
some options:

background_col = 'aliceblue'
title_col = '#0D5F8A'
mark_col = '#335120'
background_text_col = '#6EC3C1'

background_col = 'aliceblue'
title_col = '#238795'
mark_col = '#952387'
background_text_col = '#879523'

background_col = 'aliceblue'
title_col = '#284E60'
mark_col = '#63AAC0'
background_text_col = '#F99B45'

"""

background_col = 'aliceblue'
title_col = '#004B99'
mark_col = '#99004B'
background_text_col = '#4B9900'

In [321]:
#color options https://altair-viz.github.io/user_guide/generated/core/altair.ColorName.html
#https://www.sessions.edu/color-calculator/

chart = alt.Chart(mood_df,title="Mood this week").mark_circle(size=60,color=mark_col).encode(
    x=alt.X('mood1',title='Negative \u21e8 Positive'), #change the axis title
    y=alt.Y('mood2',title='Sleepy \u21e8 Alert'), #have to use unicode symbols; \u21e8 is thick right arrow
)

#put text on background
watermark1 = alt.Chart(pd.DataFrame([1])).mark_text(
    align='left', dx=-150, dy=-75, fontSize=30, text='Stress', color=background_text_col
).encode(
    opacity=alt.value(0.5)
)
watermark2 = alt.Chart(pd.DataFrame([1])).mark_text(
    align='right', dx=150, dy=-75, fontSize=30, text='Excitement', color=background_text_col
).encode(
    opacity=alt.value(0.5)
)
watermark3 = alt.Chart(pd.DataFrame([1])).mark_text(
    align='left', dx=-150, dy=75, fontSize=30, text='Depressed', color=background_text_col
).encode(
    opacity=alt.value(0.5)
)
watermark4 = alt.Chart(pd.DataFrame([1])).mark_text(
    align='right', dx=150, dy=75, fontSize=30, text='Relaxed', color=background_text_col
).encode(
    opacity=alt.value(0.5)
)

#combine plot and background text
final_chart = alt.layer(chart,watermark1,watermark2,watermark3,watermark4).configure(background=background_col).configure_axis(
    titleFontSize=15,titleColor=title_col).configure_title(fontSize=25,color=title_col).properties(width=400,height=400)
final_chart

alt.LayerChart(...)

In [322]:
alt.Chart(mood_df,title="Mood this week").mark_circle(size=100,color=mark_col).encode(
    x=alt.X('short_date',title='Date'), #change the axis title
    y=alt.Y('mood1',title='Mood')
).properties(width=400,height=400).configure(background=background_col).configure_title(
    fontSize=25,color=title_col).configure_axis(
    titleFontSize=15,titleColor=title_col)

alt.Chart(...)

In [323]:
#From Philip

alt.Chart(conc_df,title="Concentration this week").mark_line(color=mark_col).encode(
    x=alt.X('short_date',title='Date'), #change the axis title
    y=alt.Y(field='conc_label',type='nominal',sort=alt.EncodingSortField(field='conc',order='descending'),title='Concentration'),
).properties(width=400,height=400).configure(background=background_col).configure_title(
    fontSize=25,color=title_col).configure_axis(
    titleFontSize=15,titleColor=title_col)


alt.Chart(...)

In [324]:
#From Frank
"""
alt.Chart(gd_df,title="Will I Have a Good Day Tomorrow?").mark_line(color=mark_col).encode(
    x=alt.X('short_date',title='Date'), #change the axis title
    y=alt.Y(field='conc_label',type='nominal',sort=alt.EncodingSortField(field='conc',order='descending'),title='Concentration'),
).properties(width=400,height=400).configure(background=background_col).configure_title(
    fontSize=25,color=title_col).configure_axis(
    titleFontSize=15,titleColor=title_col)
"""
alt.Chart(gd_df, title = 'Response to "Will I Have a Good Day Tomorrow?"').mark_rect().encode(
    x=alt.X('short_date',title='Date'),
    y=alt.Y(field='good_day',type='nominal',sort=alt.EncodingSortField(field='conc',order='descending'),title='Good Day Tomorrow'),
    color = alt.Color('good_day', legend = None, scale = alt.Scale(
            domain=['Yes', 'No'],
            range=['green', '#99004B']))
).properties(width=400,height=200).configure(background=background_col).configure_title(
    fontSize=25,color=title_col).configure_axis(
    titleFontSize=15,titleColor=title_col)

alt.Chart(...)